## **Utilisation simple des identifieurs de langues** (rev2) **avec les :**
>### **- '*Sparse*' Bag Of Words**
>### **- Tokenisations BERT ou Tiktoken**
>### **- Classificateurs Naïve Bayes et Gradiant Boosting**

#### **Choix du tokenizer**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import joblib
from sklearn.feature_extraction.text import CountVectorizer

# Choix de la Tokenisation (False = BERT, True Tiktoken)
titoken_tokenization = True

# Ce parametre permet éventuellement d'équilibrer de nombre de phrase par langue.
# Si ce parametre est très grand, tout le corpus sera lu. 
nb_phrase_lang = 500000

import warnings
warnings.filterwarnings('ignore')

#### **Lectures des phrases de "sentences.csv", et de leur étiquette "Langue" pour les langues sélectionnées**

In [2]:
# Ouvrir le fichier d'entrée en mode lecture
def create_lang_df(path):
    df = pd.read_csv(path, index_col ='id')
    return df

df_big = create_lang_df('../data/multilingue/sentences.csv')
lan_code = ['eng','fra','deu','spa','ita']
df = pd.DataFrame(columns=df_big.columns)
for i in range(len(lan_code)):
    df= pd.concat([df, df_big[df_big['lan_code']==lan_code[i]].iloc[:nb_phrase_lang]])
df = df.sample(frac=1, random_state=3).reset_index(drop=True)
n_rows = len(df)
print('Nombre de lignes de sentence.csv:',n_rows)
nb_phrases_lang =[]
for l in lan_code:
    nb_phrases_lang.append(sum(df['lan_code']==l))
print("Nombre de phrases par langue ",lan_code,":",nb_phrases_lang)
display(df.head(10))
display(df.tail(10))

Nombre de lignes de sentence.csv: 1750000
Nombre de phrases par langue  ['eng', 'fra', 'deu', 'spa', 'ita'] : [350000, 350000, 350000, 350000, 350000]


,lan_code,sentence
0,eng,She is afraid of death.
1,ita,Indovina cosa scelgo io.
2,spa,"¿Puedo ayudarlo? ""No, gracias. Solo estoy mira..."
3,ita,Io non sono una fricchettona!
4,deu,"Es sind schon fast 10 Jahre vergangen, aber du..."
5,spa,Creía que me quería.
6,eng,This school sets high moral standards for pupils.
7,eng,"Man is judged by his courage, woman by her charm."
8,fra,Je mange des pruneaux sucrés.
9,fra,J'ai écrit une chanson pour toi.


,lan_code,sentence
1749990,deu,Es geschieht heutzutage ja so viel in unserer ...
1749991,spa,El almuerzo está preparado.
1749992,eng,I've seen enough.
1749993,ita,Hanno accelerato il passo.
1749994,fra,Elle en pince pour ce garçon.
1749995,deu,"Wer von uns wünschte nicht manchmal, dass er d..."
1749996,ita,No! Io odio i broccoli!
1749997,fra,Tu seras tuée !
1749998,fra,Tom aurait dû manger plus.
1749999,eng,He took the video to a local TV station.


#### **Selection du tokenizer** en fonction de la variable titoken_tokenization

In [3]:
# Selection du tokenizer
if titoken_tokenization:
    import tiktoken
    tokenizer = tiktoken.get_encoding("cl100k_base")
else:
    from transformers import BertTokenizerFast
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased')

#### **Préparation de la vectorisation par CountVectorizer**
#### **Création de la fonction de création d'un Bags  Of Worlds pour les phrases à identifier**

In [4]:
# Les 2 fonctions suivantes sont nécéssaires afin de sérialiser ces parametre de CountVectorizer
# et ainsi de sauvegarder le vectorizer pour un un usage ultérieur sans utiliser X_train pour  le réinitialiser
def custom_tokenizer(text):
    tokens = tokenizer.encode(text)  # Cela divise le texte en mots
    return tokens

def custom_preprocessor(text):
    return text

# CountVectorizer a une liste de phrase en entrée.
# Cette fonction met les données d'entrée dans le bon format
def format_to_vectorize(data):
    X_tok = []
    if "DataFrame" in str(type(data)):sentences = df.tolist()
    elif "str" in str(type(data)):
        sentences =[data]
    else: sentences = data
                          
    for sentence in sentences:
        X_tok.append(sentence) # ('¤'.join([tokenizer.decode([ids]) for ids in tokenizer.encode(sentence)])+'¤')
    return X_tok

def create_BOW(data):
    global vectorizer
    
    X_tok = format_to_vectorize(data)
    X = vectorizer.transform(X_tok)
    return X

#### **Chargement du vectorizer**

In [5]:
def load_vectorizer():
    global dict_token, dict_ids, nb_token
    
    if titoken_tokenization: path = '../data/vectorizer_tiktoken.pkl'
    else: path = '../data/vectorizer_BERT.pkl'
    vectorizer = joblib.load(path)
    dict_token = {tokenizer.decode([cle]): cle for cle, valeur in vectorizer.vocabulary_.items()}
    dict_ids = {cle: tokenizer.decode([cle]) for cle, valeur in vectorizer.vocabulary_.items()} #dict_ids.items()}
    nb_token = len(vectorizer.vocabulary_)
    return vectorizer

vectorizer = load_vectorizer()

#### **Choix du nom du fichier du classifieur sauvegardé**

In [6]:
def get_file_name(titoken_tokenization, classifier):
    if titoken_tokenization:
        return "id_lang_tiktoken_"+classifier+"_sparse.pkl"
    else:
        return "id_lang_BERT_"+classifier+"_sparse.pkl"

#### **Chargement du classificateur entrainé avec l'algorithme Naïve Bayes**

In [7]:
from sklearn import naive_bayes

# Chargement du classificateur sauvé
clf_nb = joblib.load("../data/"+get_file_name(titoken_tokenization,"nb"))


#### **Chargement du classificateur entrainé avec l'algorithme Gradiant Boosting**

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

# Chargement du classificateur sauvé
clf_gb = joblib.load("../data/"+get_file_name(titoken_tokenization,"gb"))
######### dict_ids, decoded_keys = load_dict_token() ######### 

#### **Definition de fonctions identificateur de langue avec Naive Bayes**

In [9]:
def lang_id_nb(sentences):
    if "str" in str(type(sentences)):
        return clf_nb.predict(create_BOW(sentences))[0]
    else: return clf_nb.predict(create_BOW(sentences))

def lang_id_gb(sentences):
    if "str" in str(type(sentences)):
        return clf_gb.predict(create_BOW(sentences))[0]
    else:
        return clf_gb.predict(create_BOW(sentences))        

#### **Exemples d'utilisation**

In [10]:
import random
# Instanciation d'exemples

sentence_no = random.sample(range(len(df)),5)

exemples = ["france is often snowy during spring , and it is relaxing in january .",
           "elle adore les voitures très luxueuses, et toi ?",
           "she loves very luxurious cars, don't you?",
           "vamos a la playa",
           "Ich heiße Keyne, und das ist wunderbar",
           "she loves you, mais elle te hait aussi, and das ist traurig", # Attention à cette phrase trilingue
           "I ate caviar", 
           "Vogliamo visitare il Colosseo e nuotare nel Tevere.",
            df['sentence'].iloc[sentence_no[0]],
            df['sentence'].iloc[sentence_no[1]],
            df['sentence'].iloc[sentence_no[2]],
            df['sentence'].iloc[sentence_no[3]],
            df['sentence'].iloc[sentence_no[4]],
          ]
lang_exemples = ['eng','fra','eng','spa','deu','e,f,d','en','ita',df['lan_code'].iloc[sentence_no[0]],df['lan_code'].iloc[sentence_no[1]],df['lan_code'].iloc[sentence_no[2]],
                 df['lan_code'].iloc[sentence_no[3]],df['lan_code'].iloc[sentence_no[4]]]
print('no\t lang\t Phrase')                            
for i in range(len(exemples)):
    print(i,'-\t',lang_exemples[i],'\t',exemples[i])

no	 lang	 Phrase
0 -	 eng 	 france is often snowy during spring , and it is relaxing in january .
1 -	 fra 	 elle adore les voitures très luxueuses, et toi ?
2 -	 eng 	 she loves very luxurious cars, don't you?
3 -	 spa 	 vamos a la playa
4 -	 deu 	 Ich heiße Keyne, und das ist wunderbar
5 -	 e,f,d 	 she loves you, mais elle te hait aussi, and das ist traurig
6 -	 en 	 I ate caviar
7 -	 ita 	 Vogliamo visitare il Colosseo e nuotare nel Tevere.
8 -	 deu 	 Drei von Toms Freunden besuchten Marias Feier.
9 -	 spa 	 ¿Qué piensas que hará ella?
10 -	 deu 	 Ich ziehe es vor, zu lesen, anstatt zu schreiben.
11 -	 spa 	 Ella le saluda cada mañana.
12 -	 eng 	 How many specimens can the public see at the Smithsonian Institution?


In [11]:
# Affichage des prédictions
print("Langue réelle\tPréd. Naive B.\tPréd. Grad. B.\tPhrase")
for i in range(len(exemples)):
    print(lang_exemples[i]+'\t\t'+lang_id_nb(exemples[i])+'\t\t'+lang_id_gb(exemples[i])+'\t\t'+exemples[i])


Langue réelle	Préd. Naive B.	Préd. Grad. B.	Phrase
eng		eng		eng		france is often snowy during spring , and it is relaxing in january .
fra		fra		fra		elle adore les voitures très luxueuses, et toi ?
eng		eng		eng		she loves very luxurious cars, don't you?
spa		spa		spa		vamos a la playa
deu		deu		deu		Ich heiße Keyne, und das ist wunderbar
e,f,d		fra		deu		she loves you, mais elle te hait aussi, and das ist traurig
en		ita		eng		I ate caviar
ita		ita		ita		Vogliamo visitare il Colosseo e nuotare nel Tevere.
deu		deu		spa		Drei von Toms Freunden besuchten Marias Feier.
spa		spa		spa		¿Qué piensas que hará ella?
deu		deu		deu		Ich ziehe es vor, zu lesen, anstatt zu schreiben.
spa		spa		spa		Ella le saluda cada mañana.
eng		eng		eng		How many specimens can the public see at the Smithsonian Institution?


> **Recherche des phrases mal classées par Naive Bayes**

In [12]:
n_bad_max = 30
n_bad = 0
print("\tN°Ligne\tL. réelle\tPréd. Naive B.\tPhrase")
for i in range(len(df)):
    if (n_bad<n_bad_max):
        if (df['lan_code'].iloc[i] != lang_id_nb(df['sentence'].iloc[i])):
            n_bad +=1
            print(n_bad,'\t',i,'\t-',df['lan_code'].iloc[i],'\t\t'+lang_id_nb(df['sentence'].iloc[i]),'\t\t'+
                  df['sentence'].iloc[i]," (proba={:.2f}".format(max(clf_nb.predict_proba(create_BOW([df['sentence'].iloc[i]]))[0]))+")")

	N°Ligne	L. réelle	Préd. Naive B.	Phrase
1 	 185 	- ita 		spa 		Me lo traduci?  (proba=0.86)
2 	 209 	- spa 		ita 		Admiro tu talento.  (proba=0.72)
3 	 921 	- ita 		fra 		Ce la caveremo.  (proba=0.79)
4 	 1743 	- ita 		spa 		Marie era esigente.  (proba=0.82)
5 	 1841 	- ita 		spa 		Lo odio, lo odio, lo odio.  (proba=0.82)
6 	 1970 	- spa 		ita 		Tom odia a Mary.  (proba=0.71)
7 	 2101 	- eng 		ita 		Magda marries a Spaniard.  (proba=0.67)
8 	 2365 	- spa 		ita 		Aflójate la corbata.  (proba=0.60)
9 	 2659 	- eng 		deu 		Tom is an unrepentant sinner.  (proba=0.97)
10 	 3111 	- fra 		spa 		Tom a visité Londres.  (proba=0.90)
11 	 3527 	- fra 		ita 		L'ennemi bombarda l'usine.  (proba=0.66)
12 	 3577 	- ita 		spa 		Vengo.  (proba=0.80)
13 	 3680 	- ita 		spa 		Ero a Boston.  (proba=0.76)
14 	 4607 	- ita 		spa 		Era esigente.  (proba=0.89)
15 	 5604 	- ita 		spa 		Crede a Babbo Natale.  (proba=0.64)
16 	 5973 	- ita 		spa 		Tocca a me?  (proba=0.83)
17 	 6035 	- fra 		ita 		Voici Tatoeba